In [1]:
#!pip install pymongo==4.8.0

In [1]:
print("hello")

hello


In [1]:
from pymongo import MongoClient 
from pymongo.errors import PyMongoError



In [3]:
client = MongoClient('localhost', 27017)
mydatabase = client['ubuntu4'] 
mycollection = mydatabase['users']
record = {
'title': 'MongoDB and Python', 
'description': 'MongoDB is no SQL database', 
'tags': ['mongodb', 'database', 'NoSQL'], 
'viewers': 104 
}
rec = mydatabase.myTable.insert_one(record) 

In [4]:
client = MongoClient('mongodb://localhost:27017')
db = client.test
coll = db.test

def update_device(device_num, readingGram, lastRead):
    return coll.update_one({'device': device_num}, {'$set': {'readingGram': readingGram, 'lastRead': lastRead}})

print(update_device(0, 100, 123))

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)


In [2]:
import xml.etree.ElementTree as ET
input_files = ["users", "badges", "tags", "posts", "posttags", "comments", "dummy"]

db_config = {
    'host': 'localhost',
    'port': 27017,
    'database': 'ubuntu4'
}

data_directory = 'C:\\Users\\Muhammad Raees\\OneDrive - rit.edu\\Python_Projects\\BigData\\small_data\\'

chunk_size = 5

print("hello")

hello


In [3]:
# method to connect with database
def connect():
    try:
        client = MongoClient(host=db_config['host'], 
                         port=db_config['port'])
        return client[db_config['database']]  # Return the database object
    except PyMongoError as e:
        print(f"Error connecting to MongoDB: {e}")
        return None


In [4]:
def fetch_existing_ids(collection, key):
    # collection = 'users', key = 'Id'
    db = connect()
    existing_data = db[collection].find({}, {'_id': 1, key: 1})
    return {str(doc[key]): doc['_id'] for doc in existing_data}

def extract_ids(chunk_data, key):
    # key = 'Id'
    return {entry[key] for entry in chunk_data if key in entry}

def filter_and_replace_ids(chunk_data, existing_ids, key):
    valid_chunk_data = []

    for entry in chunk_data:
        test_id = str(entry.get(key))
        # print('test_id:', test_id)  # Debug print to check the value of test_id

        # Ensure test_id is a string for comparison
        if test_id is not None and str(test_id) in existing_ids:
            # print('test_id in existing_ids:', test_id)  # Debug print to confirm matching ID
            # Replace UserId with corresponding _id
            entry[key] = existing_ids[str(test_id)]  # Convert test_id to string for dictionary lookup
            valid_chunk_data.append(entry)

    return valid_chunk_data


In [5]:
def insert_posts(input_file, max_chunks=5):
    db = connect()
    collection = db['posts']
    input_file = data_directory + input_file
    context = ET.iterparse(input_file, events=("start", "end"))
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []

    for event, elem in context:
        if elem.tag == 'row':
            if elem.get('Id') is not None and elem.get('OwnerUserId') is not None: 
                # get posts data to insert into posts
                tags_string = elem.get('Tags')
                tag_list = []
                if tags_string:
                    tags = [tag.strip('>') for tag in tags_string.split('<') if tag.strip()]
                    tag_list = [tag for tag in tags if tag]
                document = {
                "Id": elem.get('Id'),
                "ParentId": elem.get('ParentId'),
                "OwnerUserId": elem.get('OwnerUserId'),
                "AcceptedAnswerId": elem.get('AcceptedAnswerId'),
                "Title": elem.get('Title'),
                "Body": elem.get('Body'),
                "Score": elem.get('Score'),
                "ViewCount": elem.get('ViewCount'),
                "CreationDate": elem.get('CreationDate'),
                "Tags": tag_list,
                'Comments': []
                }
                
                chunk_data.append(document)
                elements_in_chunk += 1

            # If chunk is full, save the chunk and reset
            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                # Insert into db
                existing_ids = fetch_existing_ids('users', 'Id')
                print(existing_ids)
                valid_chunk_data = filter_and_replace_ids(chunk_data, existing_ids, 'OwnerUserId')

                # Insert valid data into the collection
                if valid_chunk_data:
                    collection.insert_many(valid_chunk_data)
                chunk_data = []  
                elements_in_chunk = 0

            # Clear the element to free up memory
            elem.clear()
    
    # Processing the remaining data in final chunk
    if elements_in_chunk > 0:
        chunk_count += 1
        # Insert into db
        existing_ids = fetch_existing_ids('users', 'Id')
        print(existing_ids)

        valid_chunk_data = filter_and_replace_ids(chunk_data, existing_ids, 'OwnerUserId')

        # Insert valid data into the collection
        if valid_chunk_data:
            collection.insert_many(valid_chunk_data)
        chunk_data = []  
        elements_in_chunk = 0

    print(f"Processed {chunk_count} chunks for about {chunk_size} elements.")

insert_posts("Posts.xml")    

{'-1': ObjectId('67148c60456d2db8d608ad98'), '2': ObjectId('67148c60456d2db8d608ad99'), '3': ObjectId('67148c60456d2db8d608ad9a'), '4': ObjectId('67148c60456d2db8d608ad9b'), '5': ObjectId('67148c60456d2db8d608ad9c')}
Processed 1 chunks for about 5 elements.


In [6]:
def update_posts_with_comments(chunk_data):
    db = connect()
    collection = db['posts']
    
    for post_id, comment in chunk_data:
        try:
            # Update using the UserId field instead of _id
            result = collection.update_one(
                {'Id': post_id},  # Use UserId for filtering
                {'$addToSet': {'Comments': comment}},
                upsert=False  # Don't insert if user doesn't exist
            )
            if result.modified_count > 0:
                print(f"Updated post {post_id} with cemment {comment['Text']}.")
            else:
                print(f"No update needed for post {post_id}.")

        except Exception as e:
            print(f"Error updating user {post_id}: {e}")

def insert_comments(input_file, max_chunks=10):
    input_file = data_directory + input_file
    context = ET.iterparse(input_file, events=("start", "end"))
    
    chunk_count = 0
    elements_in_chunk = 0
    chunk_data = []  
    
    for event, elem in context:
        if elem.tag == 'row':
            if elem.get('Id') is not None and elem.get('PostId') is not None and elem.get('UserId') is not None:
                comment_data = {
                    "Id": elem.get('Id'),
                    "PostId": elem.get('PostId'),
                    "Score": elem.get('Score'),
                    "Text": elem.get('Text'),
                    "CreationDate": elem.get('CreationDate'),
                    "UserId": elem.get('UserId')
                }
                post_id = elem.get('PostId')
                chunk_data.append((post_id, comment_data))
                elements_in_chunk += 1

            if elements_in_chunk >= chunk_size:
                chunk_count += 1
                update_posts_with_comments(chunk_data)
                chunk_data = [] 
                elements_in_chunk = 0
            elem.clear()

    # Process any remaining data
    if elements_in_chunk > 0: 
        chunk_count += 1
        update_posts_with_comments(chunk_data)

insert_comments("Comments.xml")

No update needed for post 60036.
No update needed for post 60036.
No update needed for post 60037.
No update needed for post 56973.
No update needed for post 56973.
No update needed for post 56974.


In [7]:
def report_db_statistics():
    db = connect()
    collections = db.list_collection_names()
    for collection_name in collections:
        count = db[collection_name].count_documents({})
        print(f"Collection: {collection_name}   Documents: {count}")
report_db_statistics()

Collection: users   Documents: 5
Collection: posts   Documents: 8


In [26]:
# def cleaning_database():
#     db = connect()
#     collections = db.list_collection_names()
#     for collection_name in collections:
#         db[collection_name].drop()
#         print(f"Deleted collection: {collection_name}")
# cleaning_database()
db = connect()
docs = db.users.find()
for doc in docs:
    print(doc)

{'_id': ObjectId('67157b59d559a31da2124c79'), 'Id': '-1', 'AccountId': '-1', 'DisplayName': 'Community', 'AboutMe': '<p>Hi, I\'m not really a person.</p>\n<p>I\'m a background process that helps keep this site clean!</p>\n<p>I do things like</p>\n<ul>\n<li>Randomly poke old unanswered questions every hour so they get some attention</li>\n<li>Own community questions and answers so nobody gets unnecessary reputation from them</li>\n<li>Own downvotes on spam/evil posts that get permanently deleted</li>\n<li>Own suggested edits from anonymous users</li>\n<li><a href="https://meta.stackexchange.com/a/92006">Remove abandoned questions</a></li>\n</ul>\n', 'CreationDate': '2010-07-28T16:38:27.683', 'Reputation': '1', 'Badges': []}
{'_id': ObjectId('67157b59d559a31da2124c7a'), 'Id': '2', 'AccountId': '2', 'DisplayName': 'Geoff Dalgas', 'AboutMe': '<p>Dev #2 who helped create Stack Overflow currently working at Microsoft. I love all people willing to share their knowledge. Without community we a

In [2]:
input_string = "<postgre><database>"

# Remove the angle brackets and split the string based on the brackets
words = input_string.replace('<', ' ').replace('>', ' ').split()

# Filter out any empty strings
word_list = [word for word in words if word]

print(word_list)


['postgre', 'database']


In [33]:
print('test')

test


In [25]:
 # Import the time module, which provides time-related functions like sleep()
import time
# Loop over a range of numbers from 1 to 5 (inclusive)
for i in range(1, 6):
    # Print a loading message with a percentage that updates on the same line
    print(f"\rLoading... {i*20}%", end="")
    # Pause the execution for 1 second to simulate a loading delay
    time.sleep(1)
print("\nDone!")

Loading... 100%
Done!
